In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import math

# 数据导入

In [4]:
#周志华西瓜数据集：根据特征是否好瓜
txt_path=r".\Watermelon.txt"
with open(txt_path, encoding='utf-8') as t:
     result=[]
     lines = t.read().splitlines()
     for i in range(len(lines)):
        line=lines[i].split(',')
        del line[0]
        result.append(line)
array = np.asarray(result)
data_header = array[0,:].tolist()
print('Datas header = ',data_header)
data = array[1:,:-1] 
print(data)
labels = array[1:,-1] 
print('Labels = ', labels)
X_train = data
y_train = labels
#分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)
print("split")
print(X_train)
print("split")
print(X_test)
print("split")
print(y_train)
print("split")
print(y_test)


Datas header =  ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
[['青绿' '蜷缩' '浊响' '清晰' '凹陷' '硬滑']
 ['乌黑' '蜷缩' '沉闷' '清晰' '凹陷' '硬滑']
 ['乌黑' '蜷缩' '浊响' '清晰' '凹陷' '硬滑']
 ['青绿' '蜷缩' '沉闷' '清晰' '凹陷' '硬滑']
 ['浅白' '蜷缩' '浊响' '清晰' '凹陷' '硬滑']
 ['青绿' '稍蜷' '浊响' '清晰' '稍凹' '软粘']
 ['乌黑' '稍蜷' '浊响' '稍糊' '稍凹' '软粘']
 ['乌黑' '稍蜷' '浊响' '清晰' '稍凹' '硬滑']
 ['乌黑' '稍蜷' '沉闷' '稍糊' '稍凹' '硬滑']
 ['青绿' '硬挺' '清脆' '清晰' '平坦' '软粘']
 ['浅白' '硬挺' '清脆' '模糊' '平坦' '硬滑']
 ['浅白' '蜷缩' '浊响' '模糊' '平坦' '软粘']
 ['青绿' '稍蜷' '浊响' '稍糊' '凹陷' '硬滑']
 ['浅白' '稍蜷' '沉闷' '稍糊' '凹陷' '硬滑']
 ['乌黑' '稍蜷' '浊响' '清晰' '稍凹' '软粘']
 ['浅白' '蜷缩' '浊响' '模糊' '平坦' '硬滑']
 ['青绿' '蜷缩' '沉闷' '稍糊' '稍凹' '硬滑']]
Labels =  ['是' '是' '是' '是' '是' '是' '是' '是' '否' '否' '否' '否' '否' '否' '否' '否' '否']
split
[['青绿' '稍蜷' '浊响' '清晰' '稍凹' '软粘']
 ['浅白' '蜷缩' '浊响' '模糊' '平坦' '硬滑']
 ['浅白' '蜷缩' '浊响' '模糊' '平坦' '软粘']
 ['乌黑' '稍蜷' '浊响' '清晰' '稍凹' '软粘']
 ['乌黑' '稍蜷' '沉闷' '稍糊' '稍凹' '硬滑']
 ['浅白' '稍蜷' '沉闷' '稍糊' '凹陷' '硬滑']
 ['乌黑' '蜷缩' '浊响' '清晰' '凹陷' '硬滑']
 ['青绿' '硬挺' '清脆' '清晰' '平坦' '软粘']
 ['青绿' '蜷缩' '沉闷' '稍糊' '稍凹' '硬

# 构建决策树

In [5]:
def get_shannon_entropy(labels: np.ndarray) -> float:
    """
    输入：labels：节点中所有数据的标签，用于计算概率
    输出：shannon_entropy：香农熵
    """
    #统计并用字典存储节点中不同标签的所含样本的个数
    label_counts = Counter(labels)  # Count occurrences of each label
    total_samples = len(labels)  # Total number of samples

    shannon_entropy = 0.0
    #根据字典，计算概率和香农熵
    for label_count in label_counts.values():
        # Calculate the probability of each label
        probability = label_count / total_samples
        # Calculate entropy for each label and sum them up
        shannon_entropy -= probability * math.log2(probability)
    return shannon_entropy

# Example usage:
labels = np.array(['是', '是', '否', '否', '否'])
entropy = get_shannon_entropy(labels)
print("Shannon Entropy:", entropy)
    



Shannon Entropy: 0.9709505944546686


In [6]:
def get_conditional_entropy(datas: np.ndarray, labels: np.ndarray) -> float:
    """
    输入：datas:具体来说会是某一维特征的数据； labels：所有数据的标签用于计算概率
    输出：conditional_entropy：根据某一维特征划分的条件熵
    """
    
    #统计并存储该维特征所具有的值对应的样本数
    total_samples = len(labels)  # Total number of samples
    conditional_entropy = 0.0

    #计算条件熵（考虑条件熵和香农熵的关联）
    # Count occurrences of each unique value in datas
    value_counts = Counter(datas)

    for value, count in value_counts.items():
        # Calculate probability of the specific value
        probability = count / total_samples

        # Filter labels where datas match the specific value
        subset_labels = labels[datas == value]

        # Calculate the Shannon entropy for this subset
        subset_entropy = get_shannon_entropy(subset_labels)

        # Weighted conditional entropy
        conditional_entropy += probability * subset_entropy

    
    return conditional_entropy

# Example usage:
datas = np.array(['青绿', '乌黑', '乌黑', '青绿', '浅白'])
labels = np.array(['是', '是', '是', '否', '否'])

conditional_entropy = get_conditional_entropy(datas, labels)
print("Conditional Entropy:", conditional_entropy)

Conditional Entropy: 0.4


In [7]:
def get_best_gain(datas: np.ndarray, labels: np.ndarray) -> (float, float):
    """
    对比根据每一维特征求得的增益Gain，选出最佳特征
    输入：datas：整个数据集；labels：所有数据的标签
    输出：best_feature：当前最佳特征； best_gain：当前最佳增益
    """
    #思考信息增益的计算公式
    total_entropy = get_shannon_entropy(labels)
    best_feature = None
    best_gain = -1.0

    
    #分别对每个特征计算信息增益，并选出增益最大的特征

    for feature_index in range(datas.shape[1]):
        feature_values = datas[:, feature_index]
        unique_values = np.unique(feature_values)

        conditional_entropy = 0.0
        for value in unique_values:
            subset_indices = np.where(feature_values == value)[0]
            subset_labels = labels[subset_indices]
            probability = len(subset_indices) / len(labels)
            conditional_entropy += probability * get_shannon_entropy(subset_labels)

        information_gain = total_entropy - conditional_entropy
        if information_gain > best_gain:
            best_gain = information_gain
            best_feature = data_header[feature_index]
    print("Best Feature:", best_feature)
    print("Best Gain:", best_gain)
    return best_feature, best_gain

# Example usage:
#data = np.array([('青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'),
#                 ('乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑'),
#                ('青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑')
#                 ])

# labels = np.array(['是', '否', '否'])

# best_feature, best_gain = get_best_gain(data, labels)


In [13]:
def create_tree(datas_header: list, datas: np.ndarray, labels: np.ndarray) -> dict:
    """
    对比根据每一维特征求得的增益Gain，选出最佳特征
    输入：datas：整个数据集；labels：所有数据的标签
    输出：best_feature：当前最佳特征； best_gain：当前最佳增益
    """
    # If all labels are the same, return a leaf node with that label
    if len(set(labels)) == 1:
        return labels[0]

    # If there are no features left to split on, return the majority class
    if len(datas_header) == 0:
        label_counts = Counter(labels)
        return label_counts.most_common(1)[0][0]

    # Find the best feature to split on and its information gain
    best_feature, _ = get_best_gain(datas, labels)

    # Initialize the tree with the best_feature as the root
    tree = {best_feature: {}}

    # Remove the best_feature from the available features
    # remaining_features = [f for f in datas_header if f != best_feature]
    remaining_features = datas_header
    
    print(datas_header)
    print(best_feature)
    # Split the data into subsets based on unique values of the best_feature
    unique_values = np.unique(datas[:, datas_header.index(best_feature)])


    for value in unique_values:
        subset_indices = np.where(datas[:, datas_header.index(best_feature)] == value)
        subset_data = datas[subset_indices]
        subset_labels = labels[subset_indices]

        # Recursively create a subtree for each subset of data
        subtree = create_tree(remaining_features, subset_data, subset_labels)
        tree[best_feature][value] = subtree

    #递归出口（结束条件）写在前面，判断是否继续产生叶子节点
    
    # 结束条件1：节点中所有数据都是同类，该节点不再产生叶子节点
    
    # 结束条件2: 节点中还有不同类的标签，而没有节点能产生叶子节点，少数服从多数
    
    #递归体：判断当前最优特征，划分叶子节点，并在叶子节点的划分中不再考虑当前最优特征
    
    return tree

# data_header = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
# datas = np.array([
#    ('青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'),
#    ('乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑'),
#    ('乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑'),
#    ('青绿', '蜷缩', '沉闷', '清晰', '稍凹', '硬滑'),
#    ('浅白', '蜷缩', '浊响', '清晰', '稍凹', '硬滑')
# ])
# labels = np.array(['是', '是', '是', '否', '否'])

# Call the create_tree function
# decision_tree = create_tree(data_header, data, labels)

# Print the resulting decision tree
# import json
# print(json.dumps(decision_tree, indent=2))


In [9]:
def Predict_Results(tree_model, datas, datas_header):
    """
    测试
    """
    def predict_result(trees_model: dict, input_data: np, datas_header: list) -> str:
        cur_judge = list(trees_model.keys())[0]     
        num_feature = datas_header.index(cur_judge)     
        cur_val = input_data[num_feature]       
        cur_tree = trees_model[cur_judge]
        if type(cur_tree[cur_val]) == np.str_:
            return cur_tree[cur_val]
        return predict_result(cur_tree[cur_val], input_data, datas_header)
    
    results = np.zeros(datas.shape[0]).astype(str)
    for i,data in zip(range(datas.shape[0]), datas):
        data = data.tolist()
        result = predict_result(tree_model, data, datas_header)
        results[i] = result
    return results

def evaluation(y, results):
    num = y.shape[0]
    acc = sum(y==results) / num
    print('Accuracy = ', float(acc))

In [10]:
# 保存和读取函数
def store_tree(input_tree, filename):
    import pickle
    with open(filename, 'wb') as f:
        pickle.dump(input_tree, f)
        f.close()


def restore_tree(filename):
    import pickle
    with open(filename, 'rb') as f:
        return pickle.load(f)


In [14]:
# 创建树
data_header = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感','好瓜']
tree_model = create_tree(data_header, X_train, y_train)
print("决策树构建完成")
print(tree_model)

results = Predict_Results(tree_model, X_test, data_header)
evaluation(y_test,results)

# # 保存树模型
store_tree(tree_model, 'Watermelon.pkl')     
print("模型保存完毕")

Best Feature: 纹理
Best Gain: 0.32751829749445327
['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
纹理
Best Feature: 根蒂
Best Gain: 0.46956521111470695
['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
根蒂
Best Feature: 色泽
Best Gain: 0.2516291673878229
['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
色泽
Best Feature: 触感
Best Gain: 1.0
['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
触感
Best Feature: 触感
Best Gain: 0.7219280948873623
['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜']
触感
决策树构建完成
{'纹理': {'模糊': '否', '清晰': {'根蒂': {'硬挺': '否', '稍蜷': {'色泽': {'乌黑': {'触感': {'硬滑': '是', '软粘': '否'}}, '青绿': '是'}}, '蜷缩': '是'}}, '稍糊': {'触感': {'硬滑': '否', '软粘': '是'}}}}
Accuracy =  1.0
模型保存完毕
